# Notebook Structure

1. Data Cleaning
        1.1 Members data
        1.2 Packs data
        1.3 Merged data 
        1.4 *Dealing with nulls!!!
        
2. Feature Engineering

-- Next Notebook 


3. Feature Importance
        4.1 OLS Model on complete data
        4.2 LR Model on complete data
        4.3 DT Model on complete data
        4.4 RF Model on complete data
        4.5 Table representing MAE
        4.6 Table representing feature importance

5. Final Models per person
        5.1 LR model per person
        5.2 DT model per person
        5.3 RF model per person

# 1. Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
path = '/Users/aishwaryasingh/PuzzleSchedulingProject/puzzle-scheduling/data/'

### 1.1 Members Data

In [3]:
member_holdtime_df = pd.read_csv(path+'member_hold_times_and_packs_Feb16_2023.tsv', 
                                 sep='\t', header=None)
member_holdtime_df.columns = ['memberID', 'holdtime', 'puzzlepack']
member_holdtime_df.shape

(19733, 3)

In [4]:
member_holdtime_df.isnull().sum()

memberID      0
holdtime      0
puzzlepack    0
dtype: int64

In [5]:
# removing outliers
member_holdtime_df = member_holdtime_df[(member_holdtime_df['holdtime'] >= 0.1) & 
                                        (member_holdtime_df['holdtime'] <= 150)]

### 1.2 Packs Data

In [6]:
packs_df = pd.read_csv(path + 'packs_Feb16_2023.tsv', sep='\t', header=None)
packs_df.columns = ['pack_name', 'brand', 'piece_count', 'difficulty_rating']
packs_df.shape

(920, 4)

In [7]:
# making two coulmns for piece count
packs_df['piece_count_1'] = packs_df['piece_count'].str.split(',', expand=True)[0]
packs_df['piece_count_2'] = packs_df['piece_count'].str.split(',', expand=True)[1]
# making two columns for difficulty
packs_df['difficulty_rating_1'] = packs_df['difficulty_rating'].str.split(',', expand=True)[0]
packs_df['difficulty_rating_2'] = packs_df['difficulty_rating'].str.split(',', expand=True)[1]

In [8]:
# splitting brand name
packs_df['brand_2'] = packs_df['brand'].str.split(',', expand=True)[1]
packs_df['brand_1'] = packs_df['brand'].str.split(',', expand=True)[0]

In [9]:
# adding number of puzzles feature
packs_df['num_puzzles'] = packs_df['pack_name'].map(lambda n: 1 if (n[-1] == ' ') else 2, na_action='ignore')

In [10]:
# fixing datatype
packs_df = packs_df.astype({'piece_count_1': 'int64', 'piece_count_2': 'int64'}, errors='ignore')

In [11]:
# dropping the initial variables
packs_df.drop(['brand', 'piece_count', 'difficulty_rating'], axis=1, inplace=True)

In [12]:
packs_df.isnull().sum()

pack_name                0
piece_count_1            8
piece_count_2          181
difficulty_rating_1      4
difficulty_rating_2      4
brand_2                775
brand_1                 25
num_puzzles              0
dtype: int64

In [13]:
# making the brand_2 same as brand_1
# ------ must be done only for puzzle = 2, right?
packs_df.loc[(packs_df['brand_2'].isna()), 'brand_2'] = packs_df['brand_1']

In [14]:
packs_df[['brand_1', 'brand_2']] = packs_df[['brand_1', 'brand_2']].fillna('unknown')

In [15]:
# Some 1 puzzle packs have a second number for pieces, this seems to be legit, updating to be 2 puzzle packs
packs_df.loc[((packs_df.num_puzzles == 1) & (packs_df.piece_count_2.notna())), 'num_puzzles'] = 2

In [16]:
packs_df.piece_count_1 = packs_df.piece_count_1.astype('float').astype('Int64')
packs_df.piece_count_2 = packs_df.piece_count_2.astype('float').astype('Int64')

avg_pc1 = packs_df['piece_count_1'].median()
avg_pc2 = packs_df['piece_count_2'].median()

In [17]:
# puzzles which have no value for piece_1 is replaced with avg values
packs_df['piece_count_1'].fillna(avg_pc1, inplace=True)

In [18]:
# if 2 puzzles and second piececount is not available, replace with average
packs_df.loc[((packs_df['piece_count_2'].isna() ) &(packs_df['num_puzzles'] ==2)), 'piece_count_2'] = int(avg_pc2)

In [19]:
# puzzles with no second piece_count, make second piece count 0 
# (because num_puzzle = 2 was dealt with earlier)
packs_df.loc[(packs_df['piece_count_2'].isna()) & (packs_df['num_puzzles'] ==1), 'piece_count_2'] = int(0)

In [20]:
packs_df.isnull().sum()

pack_name              0
piece_count_1          0
piece_count_2          0
difficulty_rating_1    4
difficulty_rating_2    4
brand_2                0
brand_1                0
num_puzzles            0
dtype: int64

In [21]:
packs_df['difficulty_rating_1'].fillna('Average', inplace=True)
packs_df['difficulty_rating_2'].fillna('Average', inplace=True)

### 1.3 Merged data

In [22]:
df = member_holdtime_df.merge(packs_df, left_on='puzzlepack', right_on='pack_name', how='left')
df.head(2)

,memberID,holdtime,puzzlepack,pack_name,piece_count_1,piece_count_2,difficulty_rating_1,difficulty_rating_2,brand_2,brand_1,num_puzzles
0,member1,2.939411,Artifact Puzzles Justin Hillgrove Word Travels...,Artifact Puzzles Justin Hillgrove Word Travels...,456,548,A-Easy,Average,Artifact,Artifact,2.0
1,member1,0.998885,DaVici Puzzles Full Moon Feast DaVici Puzzles ...,DaVici Puzzles Full Moon Feast DaVici Puzzles ...,195,220,A-Easy,Hard,DaVici,DaVici,2.0


In [23]:
df.shape, member_holdtime_df.shape

((19387, 11), (19387, 3))

In [24]:
member_holdtime_df.puzzlepack.nunique(), packs_df.pack_name.nunique(), 

(968, 920)

In [25]:
df.isnull().sum()
# about 5% of the data is missing

memberID                  0
holdtime                  0
puzzlepack                0
pack_name              1085
piece_count_1          1085
piece_count_2          1085
difficulty_rating_1    1085
difficulty_rating_2    1085
brand_2                1085
brand_1                1085
num_puzzles            1085
dtype: int64

<div class="alert alert-block alert-info">
<b>Tip:</b> # Ideally we should have information about all the packs. Since we do not have this information and it is recommended that we do not drop these rows, we will impute these with the average values.
</div>

### 1.4 Dealing with Nulls!
**Methodology**

* num_puzzles --> Count number of times 'Puzzle' appears in puzzlepack
* brand_1, brand_2 --> Take word before 'Puzzle' as brand name
* difficulty_rating --> Take average at brand level from known data
* piece_count --> Take average at brand level from known data

In [26]:
missing_pack_data = df.loc[df['pack_name'].isna()][['puzzlepack', 'piece_count_1', 'piece_count_2',
                               'difficulty_rating_1', 'difficulty_rating_2', 'brand_1', 
                                'brand_2', 'num_puzzles']]
missing_pack_data.reset_index(drop=True, inplace=True)

In [27]:
# getting number of puzzles
missing_pack_data['num_puzzles'] = missing_pack_data['puzzlepack'].apply(lambda x: (x.count('Puzzles')))

In [28]:
# packs can have only 1 or two puzzles
for i in range(0, len(missing_pack_data)):
    if missing_pack_data['num_puzzles'][i] < 1:
        missing_pack_data['num_puzzles'][i] = 1
    elif missing_pack_data['num_puzzles'][i] > 2:
        missing_pack_data['num_puzzles'][i] = 2
    else: 
        pass

In [29]:
## to limit the number of brand names we will only use the brands that we know
known_brands = list(set(packs_df['brand_1'].unique()).union(set(packs_df['brand_2'].unique())))

In [30]:
# getting brand_1
for i in range(0,len(missing_pack_data)):
    words = missing_pack_data['puzzlepack'][i].split()[0:2] 
    
    if 'Puzzles' in words[1:]: #if its the second or third word
        if words[words.index('Puzzles')-1] in(known_brands):
            missing_pack_data['brand_1'][i] = words[words.index('Puzzles')-1]
        else:
            missing_pack_data['brand_1'][i] = 'unknown'

In [31]:
# getting brand_2
for i in range(0,len(missing_pack_data)):
    words = missing_pack_data['puzzlepack'][i].split()[2:]
    if 'Puzzles' in words[1:]: #if its the 4th word onwards
        if words[words.index('Puzzles')-1] in(known_brands):
            missing_pack_data['brand_2'][i] = words[words.index('Puzzles')-1]
        else:
            missing_pack_data['brand_2'][i] = 'unknown'

In [32]:
missing_pack_data[['num_puzzles', 'brand_1', 'brand_2']].isnull().sum()

num_puzzles      0
brand_1        139
brand_2        333
dtype: int64

In [33]:
# if anything still missing in num_puzzles, brands
missing_pack_data['num_puzzles'].fillna(1, inplace=True)
missing_pack_data['brand_1'].fillna('unknown', inplace=True)
missing_pack_data['brand_2'].fillna('unknown', inplace=True)

In [34]:
# brand level average piece count values -- KNOWN data
groupby_brand_pieces_1 = packs_df.groupby("brand_1")["piece_count_1"].mean()
groupby_brand_pieces_2 = packs_df.groupby("brand_2")["piece_count_2"].mean()

In [35]:
# getting piece_count_1
for i in range(0, len(missing_pack_data)):
    try:
        missing_pack_data['piece_count_1'][i] = int(groupby_brand_pieces_1[missing_pack_data['brand_1'][i]])
        # getting piece_count_2 if needed
        if missing_pack_data['num_puzzles'][i] == 1:
            missing_pack_data['piece_count_2'][i] = 0
        else:
            missing_pack_data['piece_count_2'][i] = int(groupby_brand_pieces_2[missing_pack_data['brand_2'][i]])
    except Exception as e:
        pass
#         print(e)

In [36]:
# brand level most common difficulty values -- KNOWN data
groupby_brand_diff_1 = packs_df.groupby("brand_1")["difficulty_rating_1"].agg(pd.Series.mode)
groupby_brand_diff_2 = packs_df.groupby("brand_2")["difficulty_rating_2"].agg(pd.Series.mode)

In [37]:
# getting difficulty_rating_1
for i in range(0, len(missing_pack_data)):
    try:
        missing_pack_data['difficulty_rating_1'][i] = groupby_brand_diff_1[missing_pack_data['brand_1'][i]]
        # getting difficulty_rating_2 if needed
        if missing_pack_data['num_puzzles'][i] == 1:
            missing_pack_data['difficulty_rating_2'][i] = 'Average'
        else:
            missing_pack_data['difficulty_rating_2'][i] = (groupby_brand_diff_2[missing_pack_data['brand_2'][i]])
    except Exception as e:
        pass

In [38]:
missing_pack_data.isnull().sum()

puzzlepack             0
piece_count_1          0
piece_count_2          0
difficulty_rating_1    0
difficulty_rating_2    0
brand_1                0
brand_2                0
num_puzzles            0
dtype: int64

### Making a final packs data

In [39]:
# setting order same as missing pack to concat easily
packs_df = packs_df[['pack_name', 'piece_count_1', 'piece_count_2', 'difficulty_rating_1', 
                             'difficulty_rating_2', 'brand_1', 'brand_2', 'num_puzzles']]

In [40]:
# setting same column names to concat easily
missing_pack_data = missing_pack_data.rename(columns = {'puzzlepack':'pack_name'})

In [41]:
packs_df.shape, missing_pack_data.shape

((920, 8), (1085, 8))

In [42]:
packs_updated = pd.concat([packs_df, missing_pack_data])
packs_updated.reset_index(inplace=True, drop = True)
packs_updated.shape

(2005, 8)

In [43]:
df_cleaned = member_holdtime_df.merge(packs_updated, left_on='puzzlepack', right_on='pack_name', how='left')
df_cleaned.head(2)

,memberID,holdtime,puzzlepack,pack_name,piece_count_1,piece_count_2,difficulty_rating_1,difficulty_rating_2,brand_1,brand_2,num_puzzles
0,member1,2.939411,Artifact Puzzles Justin Hillgrove Word Travels...,Artifact Puzzles Justin Hillgrove Word Travels...,456,548,A-Easy,Average,Artifact,Artifact,2
1,member1,0.998885,DaVici Puzzles Full Moon Feast DaVici Puzzles ...,DaVici Puzzles Full Moon Feast DaVici Puzzles ...,195,220,A-Easy,Hard,DaVici,DaVici,2


In [44]:
df_cleaned.drop_duplicates(inplace=True)
df_cleaned.reset_index(drop=True, inplace=True)

In [45]:
difficulty_mapping = {'A-Easy': 1, 'Average': 2, 'Hard': 3, 'Really-Hard': 4}

df_cleaned['difficulty_rating_1'] = df_cleaned['difficulty_rating_1'].map(lambda x: difficulty_mapping[x], na_action='ignore')
df_cleaned['difficulty_rating_2'] = df_cleaned['difficulty_rating_2'].map(lambda x: difficulty_mapping[x], na_action='ignore')

In [46]:
df_cleaned.shape, df.shape, member_holdtime_df.shape

((19387, 11), (19387, 11), (19387, 3))

In [47]:
df_cleaned.isnull().sum()

memberID               0
holdtime               0
puzzlepack             0
pack_name              0
piece_count_1          0
piece_count_2          0
difficulty_rating_1    0
difficulty_rating_2    0
brand_1                0
brand_2                0
num_puzzles            0
dtype: int64

# 3. Feature Engineering

### First feature is number of pieces for each difficulty d1, d2, d3, d4

In [48]:
# taking code from jon's branch
pieces_by_difficulty = []

for i, row in df_cleaned.iterrows():
    out = {
        'name': row['pack_name'],
        'pieces_d1': 0,
        'pieces_d2': 0,
        'pieces_d3': 0,
        'pieces_d4': 0,
        'num_puzzles': row['num_puzzles']
    }
    
    if not math.isnan(row['piece_count_1']):
        out[f'pieces_d{str(int(row["difficulty_rating_1"]))}'] += row['piece_count_1']
    
    if(row['num_puzzles'] == 2 and math.isnan(row['piece_count_1']) == False):
        out[f'pieces_d{str(int(row["difficulty_rating_2"]))}'] += row['piece_count_2']
        
    pieces_by_difficulty.append(out)

In [49]:
pieces_df = pd.DataFrame(pieces_by_difficulty)
pieces_df.head()

,name,pieces_d1,pieces_d2,pieces_d3,pieces_d4,num_puzzles
0,Artifact Puzzles Justin Hillgrove Word Travels...,456,548,0,0,2
1,DaVici Puzzles Full Moon Feast DaVici Puzzles ...,195,0,220,0,2
2,DaVici Puzzles Flying Frigate DaVici Puzzles H...,496,0,0,0,2
3,Liberty Puzzles Haeckel Hummingbirds Nautilus ...,0,707,0,0,2
4,DaVici Puzzles Diana Zimens City Of Cats,0,700,0,0,1


### Second Feature is counting total number of pieces at pack level

In [50]:
# making things at pack level
df_cleaned['piece_count_pack'] = 0
df_cleaned['difficulty_rating_pack'] = 0
for i in range(0, len(df_cleaned)):
    if df_cleaned['num_puzzles'][i] == 1:
        df_cleaned['piece_count_pack'][i] = df_cleaned['piece_count_1'][i]
        df_cleaned['difficulty_rating_pack'][i] = df_cleaned['difficulty_rating_1'][i]
    else:
        df_cleaned['piece_count_pack'][i] = df_cleaned['piece_count_1'][i] + df_cleaned['piece_count_2'][i]
        df_cleaned['difficulty_rating_pack'][i] = (df_cleaned['difficulty_rating_1'][i] + df_cleaned['difficulty_rating_2'][i])//2

### Third feautre is combining difficulty and piece count into a single column values

In [51]:
# taking code from madalyn's branch
pieces_df['w_pieces_diff'] = pieces_df['pieces_d1'] + pieces_df['pieces_d2']*2 + pieces_df['pieces_d3']*3 + pieces_df['pieces_d4']*4

# use box cox method to transform weighted pieces by difficulty
from sklearn.preprocessing import power_transform 
pieces_df['w_pieces_diff_transformed'] = power_transform(pieces_df[['w_pieces_diff']], method='box-cox')


In [52]:
# creat list of bin names and label names
# I just picked an arbitrary number of 20 bins, to group the piece difficulty by, this can be improved on
step = (pieces_df['w_pieces_diff_transformed'].max()-pieces_df['w_pieces_diff_transformed'].min())/20
bins = np.arange(pieces_df['w_pieces_diff_transformed'].min(), pieces_df['w_pieces_diff_transformed'].max()+step, step)
labels = ['bin' + s for s in map(str, list(range(len(bins)-1)))]

# label each puzzle by binned piece & difficulty
pieces_df['bin_label'] = pd.cut(x = pieces_df['w_pieces_diff_transformed'], bins = bins, labels = labels, include_lowest = True)
pieces_df.head()

,name,pieces_d1,pieces_d2,pieces_d3,pieces_d4,num_puzzles,w_pieces_diff,w_pieces_diff_transformed,bin_label
0,Artifact Puzzles Justin Hillgrove Word Travels...,456,548,0,0,2,1552,0.476563,bin9
1,DaVici Puzzles Full Moon Feast DaVici Puzzles ...,195,0,220,0,2,855,-0.550938,bin6
2,DaVici Puzzles Flying Frigate DaVici Puzzles H...,496,0,0,0,2,496,-1.335665,bin3
3,Liberty Puzzles Haeckel Hummingbirds Nautilus ...,0,707,0,0,2,1414,0.303165,bin9
4,DaVici Puzzles Diana Zimens City Of Cats,0,700,0,0,1,1400,0.284934,bin8


In [53]:
df_cleaned.shape, pieces_df.shape

((19387, 13), (19387, 9))

In [54]:
df_cleaned.columns

Index(['memberID', 'holdtime', 'puzzlepack', 'pack_name', 'piece_count_1',
       'piece_count_2', 'difficulty_rating_1', 'difficulty_rating_2',
       'brand_1', 'brand_2', 'num_puzzles', 'piece_count_pack',
       'difficulty_rating_pack'],
      dtype='object')

In [55]:
concat_df1 = df_cleaned[['pack_name', 'holdtime', 'piece_count_1', 'piece_count_2', 'difficulty_rating_1', 
                         'difficulty_rating_2',  'brand_1','piece_count_pack', 'difficulty_rating_pack',
                         'brand_2', 'num_puzzles']]
concat_df2 = pieces_df[['pieces_d1', 'pieces_d2', 'pieces_d3', 'pieces_d4', 'w_pieces_diff_transformed']]

In [56]:
df_features_combined = pd.concat([concat_df1, concat_df2], axis=1)

In [57]:
# setting order and saving file
df_features_combined = df_features_combined[['pack_name', 'piece_count_1', 'piece_count_2', 'difficulty_rating_1', 
                                             'difficulty_rating_2', 'brand_1', 'brand_2', 'num_puzzles', 
                                             'pieces_d1', 'pieces_d2', 'pieces_d3', 'pieces_d4','piece_count_pack',
                                             'difficulty_rating_pack', 'w_pieces_diff_transformed', 'holdtime']]

In [58]:
df_features_combined.to_csv('df_features_combined.csv', index=False)